In [1]:
import torch
torch.set_printoptions(precision=2)

import math

def print2(t):
    t = torch.clone(t)
    t[torch.abs(t) < 1e-9] = 0
    print(t[:,SEQ_LENGTH:])

def print3(t):
    t = torch.clone(t)
    t[torch.abs(t) < 1e-9] = 0
    print(t)

In [2]:
def generate_sequence(seq_length):
    path = torch.randint(0, 2, (seq_length,))  # Initialize the sequence with random binary values
    for i in range(2, seq_length):
        xor_result = path[i-1] ^ path[i-2]  # XOR of the two previous elements
        not_xor_result = 1 - xor_result     # NOT XOR (negation of XOR result)
        # Choose XOR result with 90% probability, and NOT XOR result with 10% probability
        path[i] = xor_result if torch.rand(1) < 0.9 else not_xor_result
    return path

# Example of generating a sequence of length 10
seq_length = 10
sequence = generate_sequence(seq_length)
print(sequence)

tensor([0, 1, 1, 0, 1, 1, 0, 0, 0, 0])


In [3]:
SEQ_LENGTH = 20
EXTRA_SPOTS = 8
EMBEDDING_LENGTH = SEQ_LENGTH + 1 + EXTRA_SPOTS


In [4]:
pos_embedding = torch.eye(SEQ_LENGTH)

In [5]:
SEQ_LENGTH = 6
path = generate_sequence(SEQ_LENGTH,)
path[0] = 42
path[1:] = torch.tensor([0, 1, 1, 0, 1])

EMBEDDING_LENGTH = SEQ_LENGTH + 1 + EXTRA_SPOTS
pos_embedding = torch.eye(SEQ_LENGTH)
print (pos_embedding)

path = path.unsqueeze(0).T
print (path)

prev_two_bits_placeholder = torch.zeros(SEQ_LENGTH, EXTRA_SPOTS)

full_embedding = torch.cat((pos_embedding, path, prev_two_bits_placeholder),dim=1)
print (full_embedding[:,:-8])

CURRENT_LOCATION = SEQ_LENGTH
PREV_LOCATION = SEQ_LENGTH + 1
PREV_PREV_LOCATION = SEQ_LENGTH + 2
XOR_OUTPUT_LOCATION = SEQ_LENGTH + 3
XOR_CONTRADICTED_LOCATION = SEQ_LENGTH + 4
XOR_2MOD3_CONTRADICTED_LOCATION = SEQ_LENGTH + 5
XOR_0MOD3_CONTRADICTED_LOCATION = SEQ_LENGTH + 6
XOR_1MOD3_CONTRADICTED_LOCATION = SEQ_LENGTH + 7
XOR_LIKELIHOOD_LOCATION = SEQ_LENGTH + 8

tensor([[1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.]])
tensor([[42],
        [ 0],
        [ 1],
        [ 1],
        [ 0],
        [ 1]])
tensor([[ 1.,  0.,  0.,  0.,  0.,  0., 42.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  1.]])


# attn 0

In [6]:
def get_post_attn_0(full_embedding):
    query = torch.zeros(2, SEQ_LENGTH-1, EMBEDDING_LENGTH)
    query[0, :, 1:SEQ_LENGTH] = torch.eye(SEQ_LENGTH-1)
    query[1, 0:-1, 2:SEQ_LENGTH] = torch.eye(SEQ_LENGTH-2)
    q_0 = torch.einsum("im,hom->hio",full_embedding,query)

    key = torch.zeros(2, SEQ_LENGTH-1, EMBEDDING_LENGTH)
    key[:, :, 0:SEQ_LENGTH-1] = 100 * torch.eye(SEQ_LENGTH-1)
    k_0 = torch.einsum("im,hom->hio",full_embedding,key)

    pre_sm = torch.einsum("hqd,hkd->hqk",q_0,k_0)
    attn0 = torch.softmax(pre_sm, dim=-1)

    value = torch.zeros(2, EMBEDDING_LENGTH, EMBEDDING_LENGTH)
    value[0,PREV_LOCATION,SEQ_LENGTH] = 1
    value[1,PREV_PREV_LOCATION,SEQ_LENGTH] = 1
    v_0 = torch.einsum("im,hom->hio",full_embedding,value)

    attn_out_0 = torch.einsum("hqk,hkd->hqd",attn0,v_0)
    post_attn0 = full_embedding + attn_out_0.sum(dim=0)
    return post_attn0

post_attn0 = get_post_attn_0(full_embedding)
post_attn0.shape

print3(post_attn0)

tensor([[ 1.00,  0.00,  0.00,  0.00,  0.00,  0.00, 42.00,  7.50,  7.50,  0.00,
          0.00,  0.00,  0.00,  0.00,  0.00],
        [ 0.00,  1.00,  0.00,  0.00,  0.00,  0.00,  0.00, 42.00,  7.50,  0.00,
          0.00,  0.00,  0.00,  0.00,  0.00],
        [ 0.00,  0.00,  1.00,  0.00,  0.00,  0.00,  1.00,  0.00, 42.00,  0.00,
          0.00,  0.00,  0.00,  0.00,  0.00],
        [ 0.00,  0.00,  0.00,  1.00,  0.00,  0.00,  1.00,  1.00,  0.00,  0.00,
          0.00,  0.00,  0.00,  0.00,  0.00],
        [ 0.00,  0.00,  0.00,  0.00,  1.00,  0.00,  0.00,  1.00,  1.00,  0.00,
          0.00,  0.00,  0.00,  0.00,  0.00],
        [ 0.00,  0.00,  0.00,  0.00,  0.00,  1.00,  1.00,  0.00,  1.00,  0.00,
          0.00,  0.00,  0.00,  0.00,  0.00]])


# MLP 0

In [7]:
def get_post_mlp_0(post_attn0):
    mlp_in_weight = torch.zeros(7, EMBEDDING_LENGTH)
    mlp_in_bias = torch.zeros(7)
    mlp_out_weight = torch.zeros(7, EMBEDDING_LENGTH)
    # neuron 0: start or first means conditional xor is half
    mlp_in_weight[0,:2] = 1
    mlp_out_weight[0, XOR_OUTPUT_LOCATION] = .5
    # neurons 1+2: conditional xor after first two
    mlp_in_weight[1,:2] = -1000
    mlp_in_weight[1, CURRENT_LOCATION] = 1
    mlp_in_weight[1, PREV_LOCATION] = -1
    mlp_out_weight[1, XOR_OUTPUT_LOCATION] = 1
    
    mlp_in_weight[2,:2] = -1000
    mlp_in_weight[2, CURRENT_LOCATION] = -1
    mlp_in_weight[2, PREV_LOCATION] = 1
    mlp_out_weight[2, XOR_OUTPUT_LOCATION] = 1
    
    # check if xor violated
    # neuron 3 checks if all positive.  
    mlp_in_weight[3,:3] = -1000
    mlp_in_weight[3, CURRENT_LOCATION] = 1
    mlp_in_weight[3, PREV_LOCATION] = 1
    mlp_in_weight[3, PREV_PREV_LOCATION] = 1
    mlp_in_bias[3] = -2
    mlp_out_weight[3, XOR_CONTRADICTED_LOCATION] = 1
    
    # neurons 4-6 check if exactly 1 is positive, note that two of the three mlp_in_weight are -1 and this varies for each of these neurons
    mlp_in_weight[4,:3] = -1000
    mlp_in_weight[4, CURRENT_LOCATION] = -1
    mlp_in_weight[4, PREV_LOCATION] = -1
    mlp_in_weight[4, PREV_PREV_LOCATION] = 1
    mlp_in_bias[4] = 0
    mlp_out_weight[4, XOR_CONTRADICTED_LOCATION] = 1
    
    mlp_in_weight[5,:3] = -1000
    mlp_in_weight[5, CURRENT_LOCATION] = -1
    mlp_in_weight[5, PREV_LOCATION] = 1
    mlp_in_weight[5, PREV_PREV_LOCATION] = -1
    mlp_in_bias[5] = 0
    mlp_out_weight[5, XOR_CONTRADICTED_LOCATION] = 1
    
    mlp_in_weight[6,:3] = -1000
    mlp_in_weight[6, CURRENT_LOCATION] = 1
    mlp_in_weight[6, PREV_LOCATION] = -1
    mlp_in_weight[6, PREV_PREV_LOCATION] = -1
    mlp_in_bias[6] = 0
    mlp_out_weight[6, XOR_CONTRADICTED_LOCATION] = 1

    mlp_pre = post_attn0 @ mlp_in_weight.T  + mlp_in_bias.T
    mlp_act = torch.relu(mlp_pre)
    mlp_out = mlp_act @ mlp_out_weight
    layer_0_out = post_attn0 + mlp_out
    return layer_0_out
layer_0_out = get_post_mlp_0(post_attn0)

C:\Users\rick\AppData\Local\Temp\ipykernel_19396\540679616.py:50: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3618.)
  mlp_pre = post_attn0 @ mlp_in_weight.T  + mlp_in_bias.T


In [8]:
layer_0_out.shape

torch.Size([6, 15])

# layer 1 attn - check for xor violations at the 3 different locations

In [9]:
def create_causal_mask(size):
    return torch.tril(torch.ones(size, size, dtype=torch.bool))

def apply_mask_to_attention(attn_weights, mask):
    masked_attn_weights = attn_weights.masked_fill(~mask, float('-inf'))
    return masked_attn_weights
    
def get_post_attn_1(layer_0_out):
    query = torch.zeros(3, 1, EMBEDDING_LENGTH)
    query_bias = torch.ones(3, 1, 1)
    q_1 = torch.einsum("im,hom->hio", layer_0_out, query) + query_bias

    key = torch.zeros(3, 1, EMBEDDING_LENGTH)
    for offset in range(3):
        for i in range(offset, SEQ_LENGTH, 3):  
            key[offset, :, i] = 200
    key[:, :, XOR_CONTRADICTED_LOCATION] = 100
    k_1 = torch.einsum("im,hom->hio", layer_0_out, key) 

    pre_sm_1 = torch.einsum("hqd,hkd->hqk",q_1,k_1)
    # Create a causal mask for the sequence length
    mask = create_causal_mask(SEQ_LENGTH)
    # Apply the mask to the attention weights
    masked_attn_weights = apply_mask_to_attention(pre_sm_1, mask)
    # Now you can apply softmax safely, ignoring future tokens
    attn1 = torch.softmax(masked_attn_weights, dim=-1)

    value = torch.zeros(3, EMBEDDING_LENGTH, EMBEDDING_LENGTH)
    value[0,XOR_2MOD3_CONTRADICTED_LOCATION,XOR_CONTRADICTED_LOCATION] = 1
    value[1,XOR_0MOD3_CONTRADICTED_LOCATION,XOR_CONTRADICTED_LOCATION] = 1
    value[2,XOR_1MOD3_CONTRADICTED_LOCATION,XOR_CONTRADICTED_LOCATION] = 1
    v_1 = torch.einsum("im,hom->hio",layer_0_out,value)

    attn_out_1 = torch.einsum("hqk,hkd->hqd",attn1,v_1)
    post_attn1 = layer_0_out + attn_out_1.sum(dim=0)
    return post_attn1
post_attn1 = get_post_attn_1(layer_0_out)

# mlp 1

In [10]:
def get_post_mlp_1(post_attn1):
    mlp_in_weight = torch.zeros(13, EMBEDDING_LENGTH)
    mlp_in_bias = torch.zeros(13)
    mlp_out_weight = torch.zeros(13, EMBEDDING_LENGTH)
    # neuron 0: start or first means 1/3
    mlp_in_weight[0,:2] = 1
    mlp_out_weight[0, XOR_LIKELIHOOD_LOCATION] = 1./3.
    # these terms should not use any other neurons
    mlp_in_weight[1:,:2] = -100

    # qqq3
    # neurons for spots 2, 5, 8, ...
    # exclude other spots
    mlp_in_weight[1:5, 3:SEQ_LENGTH:3] = -10
    mlp_in_weight[1:5, 4:SEQ_LENGTH:3] = -10
    
    # if my spot is activated, then my contrib should be 0.
    mlp_in_weight[1:5, XOR_2MOD3_CONTRADICTED_LOCATION] = -10
    
    # N1) f1 and f2 -> 100%
    mlp_in_weight[1, XOR_0MOD3_CONTRADICTED_LOCATION] = 1
    mlp_in_weight[1, XOR_1MOD3_CONTRADICTED_LOCATION] = 1
    mlp_in_bias[1] = -1
    mlp_out_weight[1, XOR_LIKELIHOOD_LOCATION] = 1
    # N2) f1 and !f2 
    mlp_in_weight[2, XOR_0MOD3_CONTRADICTED_LOCATION] = 1
    mlp_in_weight[2, XOR_1MOD3_CONTRADICTED_LOCATION] = -1
    mlp_in_bias[2] = 0
    mlp_out_weight[2, XOR_LIKELIHOOD_LOCATION] = .5 #.5
    # N3) !f1 and f2 
    mlp_in_weight[3, XOR_0MOD3_CONTRADICTED_LOCATION] = -1
    mlp_in_weight[3, XOR_1MOD3_CONTRADICTED_LOCATION] = 1
    mlp_in_bias[3] = 0
    mlp_out_weight[3, XOR_LIKELIHOOD_LOCATION] = .5  #.5
    # N4) !f1 and !f2 
    mlp_in_weight[4, XOR_0MOD3_CONTRADICTED_LOCATION] = -1
    mlp_in_weight[4, XOR_1MOD3_CONTRADICTED_LOCATION] = -1
    mlp_in_bias[4] = 1
    mlp_out_weight[4, XOR_LIKELIHOOD_LOCATION] = 1.0/3.0 #.5 # OK
    
    # neurons for spots 3, 6, 9, ...
    # exclude other spots
    mlp_in_weight[5:9, 2:SEQ_LENGTH:3] = -10
    mlp_in_weight[5:9, 4:SEQ_LENGTH:3] = -10
    
    # if my spot is activated, then my contrib should be 0.
    mlp_in_weight[5:9, XOR_0MOD3_CONTRADICTED_LOCATION] = -10
    
    # N5) f0 and f2 -> 100%
    mlp_in_weight[5, XOR_2MOD3_CONTRADICTED_LOCATION] = 1
    mlp_in_weight[5, XOR_1MOD3_CONTRADICTED_LOCATION] = 1
    mlp_in_bias[5] = -1
    mlp_out_weight[5, XOR_LIKELIHOOD_LOCATION] = 1
    # N6) f0 and !f2 -> 50%
    mlp_in_weight[6, XOR_2MOD3_CONTRADICTED_LOCATION] = 1
    mlp_in_weight[6, XOR_1MOD3_CONTRADICTED_LOCATION] = -1
    mlp_in_bias[6] = 0
    mlp_out_weight[6, XOR_LIKELIHOOD_LOCATION] = 1./2. # was .5 #OK 
    # N7) !f0 and f2 -> 33%
    mlp_in_weight[7, XOR_2MOD3_CONTRADICTED_LOCATION] = -1
    mlp_in_weight[7, XOR_1MOD3_CONTRADICTED_LOCATION] = 1
    mlp_in_bias[7] = 0.
    mlp_out_weight[7, XOR_LIKELIHOOD_LOCATION] = 1./3.  # was 1/3 #OK
    # N8) !f0 and !f2
    mlp_in_weight[8, XOR_2MOD3_CONTRADICTED_LOCATION] = -1
    mlp_in_weight[8, XOR_1MOD3_CONTRADICTED_LOCATION] = -1
    mlp_in_bias[8] = 1
    mlp_out_weight[8, XOR_LIKELIHOOD_LOCATION] = 1./4. 
    
    # neurons for spots 4, 7, 10, ...
    # exclude other spots
    mlp_in_weight[9:13, 2:SEQ_LENGTH:3] = -10
    mlp_in_weight[9:13, 3:SEQ_LENGTH:3] = -10
    
    # if my spot is activated, then my contrib should be 0.
    mlp_in_weight[9:13, XOR_1MOD3_CONTRADICTED_LOCATION] = -10
    
    # N9) f0 and f1 -> 100%
    mlp_in_weight[9, XOR_2MOD3_CONTRADICTED_LOCATION] = 1
    mlp_in_weight[9, XOR_0MOD3_CONTRADICTED_LOCATION] = 1
    mlp_in_bias[9] = -1
    mlp_out_weight[9, XOR_LIKELIHOOD_LOCATION] = 1
    # N10) f0 and !f1 
    mlp_in_weight[10, XOR_2MOD3_CONTRADICTED_LOCATION] = 1
    mlp_in_weight[10, XOR_0MOD3_CONTRADICTED_LOCATION] = -1
    mlp_in_bias[10] = 0
    mlp_out_weight[10, XOR_LIKELIHOOD_LOCATION] = 1./3. #OK?
    # N11) !f0 and f1 -> 33%
    mlp_in_weight[11, XOR_2MOD3_CONTRADICTED_LOCATION] = -1
    mlp_in_weight[11, XOR_0MOD3_CONTRADICTED_LOCATION] = 1
    mlp_in_bias[11] = 0
    mlp_out_weight[11, XOR_LIKELIHOOD_LOCATION] = 1./3.  #OK
    # !f0 and !f2
    mlp_in_weight[12, XOR_2MOD3_CONTRADICTED_LOCATION] = -1
    mlp_in_weight[12, XOR_0MOD3_CONTRADICTED_LOCATION] = -1
    mlp_in_bias[12] = 1
    mlp_out_weight[12, XOR_LIKELIHOOD_LOCATION] = 1./5.  #OK?

    mlp_pre = post_attn1 @ mlp_in_weight.T  + mlp_in_bias.T
    # print3 (mlp_pre)
    mlp_act = torch.relu(mlp_pre)
    # print3 (mlp_act)
    mlp_out = mlp_act @ mlp_out_weight
    
    layer_1_out = post_attn1 + mlp_out
    return layer_1_out
# layer_1_out = get_post_mlp_1(post_attn1)

# MLP 2 + Unembed combo

In [11]:
def get_out(layer_1_out, extra_returns=False):
    mlp_in_weight = torch.zeros(2, EMBEDDING_LENGTH)
    mlp_in_bias = torch.zeros(2)
    mlp_out_weight = torch.zeros(2, 2)

    mlp_in_weight[0,XOR_OUTPUT_LOCATION] = 1
    mlp_in_weight[0,XOR_LIKELIHOOD_LOCATION] = 1
    mlp_in_bias[0] = -1
    mlp_out_weight[0, 1] = .5
    mlp_out_weight[0, 0] = -.5
    
    mlp_in_weight[1,XOR_OUTPUT_LOCATION] = -1
    mlp_in_weight[1,XOR_LIKELIHOOD_LOCATION] = 1
    mlp_in_bias[1] = 0
    mlp_out_weight[1, 1] = -.5
    mlp_out_weight[1, 0] = .5
    
    mlp_pre = layer_1_out @ mlp_in_weight.T  + mlp_in_bias.T
    # print3 (mlp_pre)
    mlp_act = torch.relu(mlp_pre)
    # print3 (mlp_act)
    mlp_out = mlp_act @ mlp_out_weight
    if extra_returns:
        return mlp_out + torch.tensor([.5, .5]), mlp_act
    return mlp_out + torch.tensor([.5, .5])

# out = get_out(layer_1_out)
# print (out)

# full

In [12]:
def model(full_embedding, extra_returns = False):
    post_attn0 = get_post_attn_0(full_embedding)
    layer_0_out = get_post_mlp_0(post_attn0)    
    if extra_returns:
        out0 = torch.clone(layer_0_out)
    post_attn1 = get_post_attn_1(layer_0_out)
    layer_1_out = get_post_mlp_1(post_attn1)
    if extra_returns:
        out1 = torch.clone(layer_1_out)
    if extra_returns:
        out, mlp_act = get_out(layer_1_out, extra_returns)
    else:
        out = get_out(layer_1_out)

    if extra_returns:
        return out, layer_0_out, layer_1_out, mlp_act
    
    return out
out = model(full_embedding)
print (out)

tensor([[0.50, 0.50],
        [0.50, 0.50],
        [0.33, 0.67],
        [0.62, 0.38],
        [0.40, 0.60],
        [0.33, 0.67]])


In [13]:
def embedding_V2(in_path):
    global SEQ_LENGTH
    SEQ_LENGTH = len(in_path)+1
    path = torch.zeros(SEQ_LENGTH)
    path[0] = 42
    path[1:] = in_path

    global EMBEDDING_LENGTH
    EMBEDDING_LENGTH = SEQ_LENGTH + 1 + EXTRA_SPOTS
    pos_embedding = torch.eye(SEQ_LENGTH)
    # print (pos_embedding)

    placeholder = torch.zeros(SEQ_LENGTH, EXTRA_SPOTS)
    
    full_embedding = torch.cat((pos_embedding, path.unsqueeze(1), placeholder),dim=1)

    global CURRENT_LOCATION, PREV_LOCATION, PREV_PREV_LOCATION, XOR_OUTPUT_LOCATION, XOR_CONTRADICTED_LOCATION, \
           XOR_2MOD3_CONTRADICTED_LOCATION, XOR_0MOD3_CONTRADICTED_LOCATION, XOR_1MOD3_CONTRADICTED_LOCATION, XOR_LIKELIHOOD_LOCATION

    CURRENT_LOCATION = SEQ_LENGTH                       # used in attn 0 to seed next two, used in mlp 0 to calc XOR stats
    PREV_LOCATION = SEQ_LENGTH + 1                      # created in attn 0, used in mlp 0 to calc XOR stats
    PREV_PREV_LOCATION = SEQ_LENGTH + 2                 # created in attn 0, used in mlp 0 to calc XOR stats
    XOR_OUTPUT_LOCATION = SEQ_LENGTH + 3                # calc'd in mlp 0, used in mlp 2/unembed to get prediction 
    XOR_CONTRADICTED_LOCATION = SEQ_LENGTH + 4          # calc'd in mlp 0, used in attn 1 to get contradictions 
    XOR_2MOD3_CONTRADICTED_LOCATION = SEQ_LENGTH + 5    # calc'd in attn 1, used in mlp 1 to get probability
    XOR_0MOD3_CONTRADICTED_LOCATION = SEQ_LENGTH + 6    # calc'd in attn 1, used in mlp 1 to get probability
    XOR_1MOD3_CONTRADICTED_LOCATION = SEQ_LENGTH + 7    # calc'd in attn 1, used in mlp 1 to get probability
    XOR_LIKELIHOOD_LOCATION = SEQ_LENGTH + 8            # calc'd in mlp 1, used in mlp 2/unembed to get prediction

    return full_embedding

full_embedding = embedding_V2(torch.tensor([0, 1, 1, 0, 1, 1,  1]))
print (full_embedding)

tensor([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 42.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.]])


In [14]:
path = torch.tensor([0, 1, 1, 0, 1, 1,  1])
full_embedding = embedding_V2(path)
out = model(full_embedding)
print (out[-1][0].item())

# math.isclose(a, b, abs_tol=1e-5)

0.6666666865348816


In [15]:
for k in range(15):
    print (f"\n\n\n---starting {k} -----")

    all_combinations = torch.tensor([[int(x) for x in format(i, f'0{k}b')] for i in range(2**k)])
    
    result_tensor = all_combinations[:64]
    result_tensor
    
    # C/D are center
    # A/B are next
    # X/Y are next
    # Q is dead
    starts = ["C", "D", "A", "B", "X", "Y","Q"]
    
    
    start_string_to_final_step = {}
    
    eval_count = 0
    
    for path_id, path in enumerate(all_combinations):
        q_count = 0
        to0 = 0
        to1 = 0
        for start in starts[:-1]:
            spot = start
            prob_weight = 1
    
            # print ("\nstarting", spot, path)
            for step in path:
                if spot == "C" or spot == "D":
                    if step == 0:
                        spot = "B"
                    elif step == 1:
                        spot = "A"
                    prob_weight *= .5
                elif spot == "A":
                    if step == 0:
                        spot = "Y"
                    elif step == 1:
                        spot = "X"
                    prob_weight *= .5
                elif spot == "B":
                    if step == 0:
                        spot = "X"
                    elif step == 1:
                        spot = "Y"
                    prob_weight *= .5
                elif spot == "X":
                    if step == 0:
                        spot = "C"
                    elif step == 1:
                        spot = "Q"
                        q_count+=1
                        break
                elif spot == "Y":
                    if step == 0:
                        spot = "Q"
                        q_count+=1
                        break
                    elif step == 1:
                        spot = "C"
            # print ("ending", spot, path)
            
            # key = tuple(path[:-1].tolist()) 
            # if key not in all_keys:
            #     all_keys[key] = StringToFinal()
    
            # print (path, start, spot)
            if spot != "Q":
                if spot == "X":
                    # all_keys[key].to0 += 2
                    to0 += 2 * prob_weight
                elif spot == "Y":
                    # all_keys[key].to1 += 2
                    to1+= 2 * prob_weight
                else:
                    # all_keys[key].to0 += 1
                    # all_keys[key].to1 += 1
                    to1+=1 * prob_weight
                    to0+=1 * prob_weight
    
            # else:
            #     # all_keys[key].Q += 1
            #     q_count+=1
    
        if to0 + to1 > 0:
            target_value = to0/(to0 + to1)
    
            full_embedding = embedding_V2(path)
            out = model(full_embedding)
            print(eval_count, path_id, out[-1][0].item(), target_value, to0, to1, 2*q_count, path)

            assert math.isclose(out[-1][0].item(), target_value, abs_tol=1e-5)
            eval_count += 1
        # else:
        #     print (path_id)
    
    
    # print ("q_count", q_count)
    
    # for key in all_keys:
    #     if all_keys[key].to0 + all_keys[key].to1 >= 1:
    #         print (key, all_keys[key].to0, all_keys[key].to1, all_keys[key].Q)




---starting 0 -----
0 0 0.5 0.5 3.0 3.0 2 tensor([0])



---starting 1 -----
0 0 0.5 0.5 3.0 3.0 2 tensor([0])
1 1 0.5 0.5 3.0 3.0 2 tensor([1])



---starting 2 -----
0 0 0.6666666865348816 0.6666666666666666 2.0 1.0 4 tensor([0, 0])
1 1 0.3333333134651184 0.3333333333333333 1.0 2.0 4 tensor([0, 1])
2 2 0.3333333134651184 0.3333333333333333 1.0 2.0 4 tensor([1, 0])
3 3 0.6666666865348816 0.6666666666666666 2.0 1.0 4 tensor([1, 1])



---starting 3 -----
0 0 0.625 0.625 1.25 0.75 4 tensor([0, 0, 0])
1 1 0.25 0.25 0.25 0.75 8 tensor([0, 0, 1])
2 2 0.25 0.25 0.25 0.75 8 tensor([0, 1, 0])
3 3 0.625 0.625 1.25 0.75 4 tensor([0, 1, 1])
4 4 0.75 0.75 0.75 0.25 8 tensor([1, 0, 0])
5 5 0.375 0.375 0.75 1.25 4 tensor([1, 0, 1])
6 6 0.375 0.375 0.75 1.25 4 tensor([1, 1, 0])
7 7 0.75 0.75 0.75 0.25 8 tensor([1, 1, 1])



---starting 4 -----
0 0 0.6000000238418579 0.6 0.75 0.5 4 tensor([0, 0, 0, 0])
1 1 0.3333333134651184 0.3333333333333333 0.25 0.5 6 tensor([0, 0, 0, 1])
2 2 0.0 0.0 0 0.25 10 

# stuff to throw into provided colab

In [16]:
transformer_inputs = torch.load( "comp_mech_inputs.pth")

In [17]:
transformer_inputs.shape

torch.Size([436, 10])

In [18]:
all0streams = torch.zeros(436, 11, 20)
all1streams = torch.zeros(436, 11, 20)
all_act_streams = torch.zeros(436, 11, 2)

for input_index in range(436):
    input = transformer_inputs[input_index]
    full_embedding = embedding_V2(input)
    out, post0stream, post1stream,mlp_act = model(full_embedding, extra_returns = True)
    print (mlp_act.shape)
    all0streams[input_index] = post0stream
    all1streams[input_index] = post1stream
    all_act_streams[input_index] = mlp_act

torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])
torch.Size([11, 2])


In [19]:
all1_and_act = torch.cat((all1streams, all_act_streams), dim=-1)
all1_and_act.shape

torch.Size([436, 11, 22])

In [20]:
torch.save(all0streams,"all0.pth")
torch.save(all1streams,"all1.pth")
torch.save(all_act_streams,"all_act_streams.pth")
torch.save(all1_and_act,"all1_and_act.pth")

In [21]:
print3(all1streams[0])

tensor([[ 1.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00,
          0.00, 42.00,  4.18,  4.18,  0.50,  0.00,  0.00,  0.00,  0.00,  0.33],
        [ 0.00,  1.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00,
          0.00,  1.00, 42.00,  4.18,  0.50,  0.00,  0.00,  0.00,  0.00,  0.33],
        [ 0.00,  0.00,  1.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00,
          0.00,  0.00,  1.00, 42.00,  1.00,  0.00,  0.00,  0.00,  0.00,  0.33],
        [ 0.00,  0.00,  0.00,  1.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00,
          0.00,  0.00,  0.00,  1.00,  0.00,  1.00,  1.00,  0.00,  0.00,  0.50],
        [ 0.00,  0.00,  0.00,  0.00,  1.00,  0.00,  0.00,  0.00,  0.00,  0.00,
          0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  1.00,  0.00,  0.00,  0.33],
        [ 0.00,  0.00,  0.00,  0.00,  0.00,  1.00,  0.00,  0.00,  0.00,  0.00,
          0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  1.00,  0.00,  0.00,  0.00],
        [ 0.00,  0.00,  0.00,  0.00,  0.00,  0

# example for writeup

In [22]:
torch.set_printoptions(linewidth=200)  # Set a large line width

all0streams = torch.zeros(436, 11, 20)
all1streams = torch.zeros(436, 11, 20)
all_act_streams = torch.zeros(436, 11, 2)

for input_index in range(12,13):
    input = transformer_inputs[input_index][:7]
    full_embedding = embedding_V2(input)
    out, post0stream, post1stream,mlp_act = model(full_embedding, extra_returns = True)


    print (input_index)
    print3(post1stream)

12
tensor([[ 1.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00, 42.00,  5.62,  5.62,  0.50,  0.00,  0.00,  0.00,  0.00,  0.33],
        [ 0.00,  1.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  1.00, 42.00,  5.62,  0.50,  0.00,  0.00,  0.00,  0.00,  0.33],
        [ 0.00,  0.00,  1.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  1.00, 42.00,  1.00,  0.00,  0.00,  0.00,  0.00,  0.33],
        [ 0.00,  0.00,  0.00,  1.00,  0.00,  0.00,  0.00,  0.00,  1.00,  0.00,  1.00,  1.00,  0.00,  0.00,  0.00,  0.00,  0.25],
        [ 0.00,  0.00,  0.00,  0.00,  1.00,  0.00,  0.00,  0.00,  0.00,  1.00,  0.00,  1.00,  1.00,  0.00,  1.00,  0.00,  0.33],
        [ 0.00,  0.00,  0.00,  0.00,  0.00,  1.00,  0.00,  0.00,  0.00,  0.00,  1.00,  0.00,  1.00,  0.00,  1.00,  1.00,  1.00],
        [ 0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  1.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  1.00,  1.00,  0.00],
        [ 0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  0.00,  1.00,  1.00,  0.00,  0.00,  1.00,  

In [23]:
out

tensor([[0.50, 0.50],
        [0.50, 0.50],
        [0.33, 0.67],
        [0.38, 0.62],
        [0.33, 0.67],
        [1.00, 0.00],
        [0.50, 0.50],
        [0.50, 0.50]])